In [1]:
import os, json, torch, time, sys
import polyfempy as pf
import torch.optim as optim

torch.set_default_dtype(torch.float64)

In [2]:

class Simulate(torch.autograd.Function):

    @staticmethod
    def forward(ctx, solver, fric):
        # Update solver setup
        solver.set_friction_coefficient(float(fric[0]))

        # Enable caching intermediate variables in the simulation, which will be used for solve_adjoint
        solver.set_cache_level(pf.CacheLevel.Derivatives)
        # Run simulation
        solver.solve()
        # Collect transient simulation solutions
        sol = torch.tensor(solver.get_solutions())
        # Cache solver for backward gradient propagation
        ctx.solver = solver
        return sol

    @staticmethod
    @torch.autograd.function.once_differentiable
    def backward(ctx, grad_output):
        # solve_adjoint only needs to be called once per solver, independent of number of types of optimization variables
        ctx.solver.solve_adjoint(grad_output)
        # Compute initial derivatives
        grads = torch.tensor([pf.friction_coefficient_derivative(ctx.solver)])
        return None, grads


In [3]:
log_level = 3 # warning

def create_solver(args):
    solver = pf.Solver()
    solver.set_settings(json.dumps(args), True)
    solver.set_log_level(log_level)
    solver.load_mesh_from_settings()
    return solver

In [4]:
root = "."
with open(root + "/run.json", "r") as f:
    config = json.load(f)
    config["root_path"] = root + "/run.json"

# Simulation
solver1 = create_solver(config)
config["contact"]["friction_coefficient"] = 0.1
solver2 = create_solver(config)

param = torch.tensor([0.2], requires_grad=True)
solution2 = Simulate.apply(solver2, torch.tensor([0.1]))

def loss(param):
    solution1 = Simulate.apply(solver1, param)
    ndof = solution1.shape[0] / 3

    return ((torch.sum(solution1[:, -1].view(-1, 3), dim=1)[2] / ndof - torch.sum(solution2[:, -1].view(-1, 3), dim=1)[2] / ndof) ** 2) * 100000

In [ ]:
def verify_grad(input):
    param = input.clone().detach().requires_grad_(True)
    theta = torch.tensor([1.])
    l = loss(param)
    l.backward()
    grad = param.grad
    t = 1e-6
    with torch.no_grad():
        analytic = torch.sum(grad * theta)
        f1 = loss(param + theta * t)
        f2 = loss(param - theta * t)
        fd = (f1 - f2) / (2 * t)
        print(f'grad {analytic}, fd {fd} {(f1 - l) / t} {(l - f2) / t}, relative err {abs(analytic - fd) / abs(analytic):.3e}')
        print(f'f(x+dx)={f1}, f(x)={l.detach()}, f(x-dx)={f2}')
        assert(abs(analytic - fd) <= 1e-3 * abs(analytic))

# verify_grad(param)

In [ ]:
def save_to_file(x, iter, out_dir):
    sys.stdout.flush()
    outpath = os.path.join(out_dir, "iter_" + str(iter) + ".vtu")
    print("Save to", outpath)
    solver1.export_vtu(outpath, solver1.get_solutions())

In [ ]:
# optimizer = optim.LBFGS([param], lr=0.2, line_search_fn='strong_wolfe')
optimizer = optim.Adam([param], lr=0.1)

# optimization configuration
out_dir = "/Users/zizhouhuang/Desktop/polyfem-python/test/opt"
if os.path.exists(out_dir):
    os.system("rm -r " + out_dir)
os.mkdir(out_dir)

# run simulation and compute loss
def closure():
    optimizer.zero_grad()
    l = loss(param)
    l.backward()
    return l

start_time = time.time()
out_dir = os.path.join(os.getcwd(), "opt")
if os.path.exists(out_dir):
    os.system("rm -r " + out_dir)
os.mkdir(out_dir)
for iter in range(15):
    last_loss = closure()
    print("friction coefficient", param[0].detach().numpy())
    print(f'Step {iter}: energy {last_loss:.4e}, grad norm {torch.linalg.norm(param.grad):.4e}, total time {time.time() - start_time:.2f} sec')
    # save_to_file(param, iter, out_dir)

    if last_loss < 1e-6:
        break

    # Let optimizer take a step
    optimizer.step()
    if param[0] <= 0:
        with torch.no_grad():
            param[0] = 0.05

print("Optimization finished!")